# Climate variability


This second notebook showcases similar functionalities as example 1 but in addition it illustrates how **RADWave** can be used to estimate the influence of climate trends on regional wave patterns.

As in the previous notebook, we will be querying data already downloaded from the obtained from Australian Ocean Data Network portal [AODN](https://portal.aodn.org.au/). 

> We recomend to look at **RADWave** [documentation](https://radwave.readthedocs.io/en/latest/usage.html#getting-altimeter-values-from-data-providers) and the embeded video that explain how to select both a spatial bounding box and a temporal extent from the portal and how to export the file containing the `List of URLs`. This `TXT` file contains a list of `NETCDF` files for each available satellites. 


## Loading RADWave library and initialisation

We first start by importing **RADwave** library into our working space.

In [ ]:
import io
import scipy
import requests
import numpy as np 
import pandas as pd

import RADWave as rwave

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.filterwarnings('ignore', category=FutureWarning) 
warnings.filterwarnings('ignore', category=UserWarning)

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

Here we will take the list of `NetCDF` URLs downloaded from the wave data portal containing the radar altimeter data for a region located offshore New Zealand ['../dataset/IMOS_NZ.txt']

For a detail overview of the options available in the `waveAnalysis` class, you can have a look at the [waveAnalysis API](https://radwave.readthedocs.io/en/latest/RADWave.html#RADWave.altiwave.waveAnalysis).

During initialisation we also specify:

+ `bbox` the bounding box of the geographical extent for the uploaded dataset following the convention [lon min,lon max,lat min,lat max] 
+ `stime` the starting time of wave climate analysis following the convention [year, month, day] [we chose the 1st of January 1985]
+ `etime` the ending time of wave climate analysis following the convention [year, month, day] [we chose the 31st of December 2018]

Here again, we don't specify a list of particular satellites to use (`satNames` keyword) so all of them will be queried. In other words we will look at all the records from the 10 altimeters: 

JASON-2 - JASON-3 - SARAL - SENTINEL-3A - CRYOSAT-2 - ENVISAT - GEOSAT - ERS-2 - GFO - TOPEX.

In [ ]:
wa = rwave.waveAnalysis(altimeterURL='../../dataset/IMOS_NZ.txt', bbox=[175.0,177.0,-47.0,-45.0], 
                  stime=[1985,1,1], etime=[2022,12,31])

## Processing altimeters data

After class initialisation querying the actual dataset is realised by calling the `processAltimeterData` function. The description of this function is available from the [API](https://radwave.readthedocs.io/en/latest/RADWave.html#RADWave.altiwave.waveAnalysis.processAltimeterData).

The function can take some times to execute depending on the number of NETCDF files to load and the size of the dataset to query (here it should not take more than **30 s**).

> **RADWave** uses the uploaded file containing the list of URLs to query via `THREDDS` the remote data. This operation can take *several minutes* and when looking at a large region it is recommended to divide the analyse in smaller regions and download a series of URLs text file instead of the entire domain directly.

In [ ]:
#wa.processAltimeterData(max_qc=1, altimeter_pick='all', saveCSV = 'altimeterDataNZ.csv')
wa.readAltimeterData(saveCSV = 'altimeterDataNZ.csv')

Once the dataset has been queried, we can plot the altimeter data points on a map using the `visualiseData` function.

This function **plots** and **saves** in a figure the geographical coordinates of processed altimeter data.

In [ ]:
wa.visualiseData(title="Altimeter data tracks offshore New Zealand", extent=[166.,179.,-48.,-42.], 
                 addcity=None, markersize=30, zoom=8,
                 fsize=(10, 9), fsave=None)

## Computing wave regime and seasonality


### Time series

To perform wave analysis and compute the wave parameters discussed in the [documentation](https://radwave.readthedocs.io/en/latest/method.html#), we run the `generateTimeSeries` function. And we then plot these time series by calling the `plotTimeSeries` function. 

In [ ]:
timeseries = wa.generateTimeSeries()

In [ ]:
wa.plotTimeSeries(time=[1995,2022], series='H', fsize=(12, 5), fsave=None)
wa.plotTimeSeries(time=[1995,2022], series='T', fsize=(12, 5), fsave=None)

### Seasonability trends 

In addition to time series, we analyse the seasonal characteristics of the significant wave height and wave period computed from the altimeter dataset using the `computeSeasonalCharacteristics` function is provided in the [API](https://radwave.readthedocs.io/en/latest/RADWave.html#RADWave.altiwave.waveAnalysis.computeSeasonalCharacteristics).

#### Significant wave height

In [ ]:
wh_all = wa.computeSeasonalCharacteristics(series='wh', time=[1995,2022], lonlat=None, fsave=None, plot=True)

#### Wave period

Here we choose to only perform the calculation of the wave period seasonability but we don't plot the associated graphs.  

In [ ]:
T_all = wa.computeSeasonalCharacteristics(series='period', time=[1995,2022], lonlat=None, 
                                          fsave=None, plot=True)